In [91]:
from zipfile import ZipFile
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from collections import Counter
warnings.filterwarnings("ignore")
pd.set_option("display.max_colwidth",None)

In [188]:
train = pd.read_csv("Train.csv")
test = pd.read_csv("Test.csv")
extra = pd.read_csv("extra_data.csv")
sample  = pd.read_csv("SampleSubmission.csv")
defn = pd.read_csv("VariableDefinitions.csv")
ID = test["Transaction_ID"]

In [176]:
train["split"] = -777
test["split"] = -888
#extra["split"] = -999
data = pd.concat([train,test]).reset_index(drop=True)

In [177]:
try:
    del data["MERCHANT_CATEGORIZED_AT"],data["USER_ID"],data["Transaction_ID"],data["USER_AGE"]
    #del train["MERCHANT_NAME"],train["USER_AGE"]
except Exception:
    print("Already dropped")
#data["USER_AGE"] = data["USER_AGE"].fillna(data["USER_AGE"].median())
#train = pd.get_dummies(train,columns=["MERCHANT_NAME"])

In [178]:
data["PURCHASED_AT"] = pd.to_datetime(data["PURCHASED_AT"]).dt.tz_localize(None)
data["PURCHASE_DATE"] = pd.to_datetime(data["PURCHASED_AT"]).dt.day
data["PURCHASE_MONTH"] = pd.to_datetime(data["PURCHASED_AT"]).dt.month
data["PURCHASE_YEAR"] = pd.to_datetime(data["PURCHASED_AT"]).dt.year
del data["PURCHASED_AT"]

In [179]:
data = pd.get_dummies(data,columns=["MERCHANT_NAME"])

In [180]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for i in data:
    if data[i].dtypes == 'O' or data[i].dtypes ==  bool:
        data[i] = le.fit_transform(data[i])

In [181]:
train = data[data["split"] == -777]
test = data[data["split"] == -888]
extra = data[data["split"] == -999]
del train["split"],test["split"]
del extra["MERCHANT_CATEGORIZED_AS"],extra["split"],test["MERCHANT_CATEGORIZED_AS"]

In [182]:
test.shape,train.shape

((558, 448), (373, 449))

In [183]:
y = train["MERCHANT_CATEGORIZED_AS"]
del train["MERCHANT_CATEGORIZED_AS"]
X = train

In [184]:
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler()
X,y = ros.fit_resample(X,y)
X,y = np.array(X),np.array(y)

In [185]:
from sklearn.model_selection import StratifiedKFold,cross_val_score
from sklearn.metrics import log_loss,accuracy_score
sk = StratifiedKFold()
for train_index,test_index in sk.split(X,y):
    X_train,y_train = X[train_index],y[train_index]
    X_test,y_test = X[test_index],y[test_index]

In [186]:
from lightgbm import LGBMClassifier
lg = LGBMClassifier(
#     n_estimators=100,
#     boosting_type="gbdt",
#     learning_rate=0.08,#0.08
#     num_leaves=32,#32
     random_state=0
)
lg.fit(X_train,y_train)
pr_lg = lg.predict_proba(X_test)
lg.score(X_train,y_train),lg.score(X_test,y_test),log_loss(y_test,pr_lg)

(0.9975369458128078, 0.9554455445544554, 0.1820362378532085)

In [155]:
from catboost import CatBoostClassifier
cb = CatBoostClassifier(
     silent=True,
     n_estimators=60,
     learning_rate=0.8,
     depth=10,#10
     #random_state=2030
)
cb.fit(X_train,y_train)
pr_cb = cb.predict_proba(X_test)
cb.score(X_train,y_train),cb.score(X_test,y_test),log_loss(y_test,pr_cb)

(0.9889162561576355, 0.9653465346534653, 0.39544960191397543)

In [156]:
from sklearn.ensemble import BaggingClassifier
bg = BaggingClassifier(
    base_estimator=cb,
    random_state=2030
)
bg.fit(X_train,y_train)
pr_bg = bg.predict_proba(X_test)
bg.score(X_train,y_train),bg.score(X_test,y_test),log_loss(y_test,pr_bg)

(0.9901477832512315, 0.9653465346534653, 0.40320067156035083)

In [210]:
# est = [("cb",cb),("lg",lg)]
# sc = StackingClassifier(
#     estimators=est
# )
# sc.fit(X_train,y_train)
# pr_sk = sc.predict_proba(X_test)
# sc.score(X_train,y_train),sc.score(X_test,y_test),log_loss(y_test,pr_sk)

In [157]:
cols = ['Bills & Fees','Data & WiFi','Education','Emergency fund','Family & Friends','Going out','Groceries',
 'Health','Loan Repayment','Miscellaneous','Rent / Mortgage','Shopping','Transport & Fuel']

In [158]:
#test = np.array(test)
predicts = bg.predict_proba(test)
subm = pd.DataFrame(predicts,columns=cols)
subm = pd.concat([ID,subm],join="inner",axis=1)
subm.to_csv("solution.csv",index=False)